## Import dataset
#### basic metrics calculation

In [12]:
# imports

import pandas as pd
import numpy as np
import networkx as nx
from random import sample
import time

In [2]:
# load dataset (reddit social graph)
dataset = 'soc-redditHyperlinks-body.tsv' #'large_twitch_edges.csv'

data = pd.read_csv(dataset, header = 0, sep = '\t') 
data = data[[data.columns[0], data.columns[1]]]
data.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT
0,leagueoflegends,teamredditteams
1,theredlion,soccer
2,inlandempire,bikela
3,nfl,cfb
4,playmygame,gamedev


In [3]:
# Make networkx graph object

graph = nx.from_pandas_edgelist(data, data.columns[0], data.columns[1])
#graph_directed = nx.from_pandas_edgelist(data, data.columns[0], data.columns[1], create_using = nx.DiGraph)

In [4]:
# graph properties

print('Number of nodes: ', graph.number_of_nodes())
print('Number of edges: ', graph.number_of_edges())

Number of nodes:  35776
Number of edges:  124330


In [6]:
# Giant component
Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
print('Number of largest connected components: ', len(Gcc))

giant = graph.subgraph(Gcc[0])
print('Number of nodes in giant component: ', giant.number_of_nodes())

Number of largest connected components:  497
Number of nodes in giant component:  34671


In [7]:
data_undi = data.drop_duplicates()

In [70]:
print('Average Clustering coefficient: ', nx.average_clustering(graph))

Average Clustering coefficient:  0.18090101843790804


In [13]:
# input preparation for Rigel

nodes = list(giant.nodes())
N = len(nodes)
landmark_cnt = 100
landmarks = sample(nodes, landmark_cnt)  #here a landmark selection strategy could be introduced
distance_matrix = [[-1 for i in range(N)] for j in range(landmark_cnt)]
#distance_matrix = dict()
for i in range(landmark_cnt):
    #distance_matrix[i] = dict()
    start_time = time.time()
    print('landmark ', i, ' calculating...')
    node = landmarks[i]
    path_lengths = nx.single_source_shortest_path_length(giant, node)
    for j in nodes:
        distance_matrix[i][j] = int(path_lengths[j])
    print("landmark ", i," finished in --- %s seconds ---" % (time.time() - start_time))

landmark  0  calculating...


TypeError: list indices must be integers or slices, not str